
# Modular Data Scraping Tests for Google Scholar, CrossRef DOI, and SJR Metrics

This notebook tests each component separately to allow troubleshooting. Each section focuses on one aspect: 
Google Scholar author data, CrossRef DOI retrieval, and SJR metrics retrieval.

### Prerequisites
Install the following libraries before running this notebook:
```python
!pip install selenium webdriver-manager requests beautifulsoup4
```

Run each cell individually to test specific parts.


In [9]:
# Import necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import requests
import os
import time
from bs4 import BeautifulSoup


In [10]:
# Initialize the Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

## Google Scholar - Retrieve Author Profile

In [11]:
def get_author_profile(name):
    search_url = f"https://scholar.google.com/scholar?q={name.replace(' ', '+')}"
    driver.get(search_url)
    time.sleep(2)

    try:
        # Click on the first profile link
        profile_link = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'h4.gs_rt2 a'))
        )
        profile_link.click()
        time.sleep(20)

        # Basic author information
        author_data = {
            'Name': WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, 'gsc_prf_in'))
            ).text,
            'Affiliation': driver.find_element(By.CLASS_NAME, 'gsc_prf_il').text,
            'Research Interests': ", ".join([interest.text for interest in driver.find_elements(By.CSS_SELECTOR, 'a.gsc_prf_inta')])
        }
        author_df = pd.DataFrame([author_data])

        # Citation indices
        citation_data = []
        for row in driver.find_elements(By.CSS_SELECTOR, '#gsc_rsb_st tbody tr'):
            label = row.find_element(By.CSS_SELECTOR, 'td.gsc_rsb_sc1').text
            all_time = row.find_elements(By.CSS_SELECTOR, 'td.gsc_rsb_std')[0].text
            since_2019 = row.find_elements(By.CSS_SELECTOR, 'td.gsc_rsb_std')[1].text
            citation_data.append([label, all_time, since_2019])
        
        citation_df = pd.DataFrame(citation_data, columns=["Metric", "All Time", "Since 2019"])

        # Co-authors
        co_authors_data = []
        for co_author in driver.find_elements(By.CSS_SELECTOR, 'ul.gsc_rsb_a li'):
            name = co_author.find_element(By.CSS_SELECTOR, 'a').text
            affiliation = co_author.find_element(By.CLASS_NAME, 'gsc_rsb_a_ext').text
            co_authors_data.append([name, affiliation])
        
        co_authors_df = pd.DataFrame(co_authors_data, columns=["Name", "Affiliation"]) if co_authors_data else pd.DataFrame(columns=["Name", "Affiliation"])

        # Return DataFrames
        return author_df, citation_df, co_authors_df

    except Exception as e:
        print(f"Error retrieving author profile: {e}")
        # Return empty DataFrames if there was an error
        return pd.DataFrame(), pd.DataFrame(columns=["Metric", "All Time", "Since 2019"]), pd.DataFrame(columns=["Name", "Affiliation"])


## Google Scholar - Retrieve Author's Publications

### Step 1: Retrieving DOI Using Crossref


In [12]:
def get_doi_from_crossref(title):
    url = "https://api.crossref.org/works"
    params = {"query.title": title, "rows": 1}
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        items = response.json().get("message", {}).get("items", [])
        if items:
            return items[0].get("DOI", "DOI not found")
    return "DOI not found"

### Step 2: Extracting Detailed Author Publications with "Show More" Check

In [13]:
def get_detailed_author_publications():
    publications = []
    
    try:
        # Get all publication rows from the main profile page
        document_rows = driver.find_elements(By.CSS_SELECTOR, 'tr.gsc_a_tr')
        
        for row in document_rows:
            # Retrieve basic information from the list view
            title = row.find_element(By.CSS_SELECTOR, 'a.gsc_a_at').text
            citation_count = row.find_element(By.CSS_SELECTOR, 'a.gsc_a_ac.gs_ibl').text or "0"
            year = row.find_element(By.CSS_SELECTOR, 'span.gsc_a_h').text
            
            # Click to open the publication details
            row.find_element(By.CSS_SELECTOR, 'a.gsc_a_at').click()
            time.sleep(2)  # Allow time for the details page to load
            
            # Retrieve additional details on the publication detail page
            publication_details = {
                'Title': title,
                'Year': year,
                'Citation Count': citation_count,
                'DOI': get_doi_from_crossref(title)  # Get DOI directly here
            }
            
            # Try to retrieve each detail, setting a default if not found
            try:
                publication_details['Authors'] = driver.find_element(By.XPATH, '//div[text()="Auteurs"]/following-sibling::div').text
            except NoSuchElementException:
                publication_details['Authors'] = "Authors not found"

            try:
                publication_details['Source Title'] = driver.find_element(By.XPATH, '//div[text()="Revue"]/following-sibling::div').text
            except NoSuchElementException:
                publication_details['Source Title'] = "Source title not found"

            try:
                publication_details['Volume'] = driver.find_element(By.XPATH, '//div[text()="Volume"]/following-sibling::div').text
            except NoSuchElementException:
                publication_details['Volume'] = "Volume not found"

            try:
                publication_details['Issue'] = driver.find_element(By.XPATH, '//div[text()="Numéro"]/following-sibling::div').text
            except NoSuchElementException:
                publication_details['Issue'] = "Issue not found"

            try:
                publication_details['Pages'] = driver.find_element(By.XPATH, '//div[text()="Pages"]/following-sibling::div').text
            except NoSuchElementException:
                publication_details['Pages'] = "Pages not found"

            try:
                publication_details['Publisher'] = driver.find_element(By.XPATH, '//div[text()="Éditeur"]/following-sibling::div').text
            except NoSuchElementException:
                publication_details['Publisher'] = "Publisher not found"

            try:
                publication_details['Summary'] = driver.find_element(By.XPATH, '//div[text()="Description"]/following-sibling::div').text
            except NoSuchElementException:
                publication_details['Summary'] = "Summary not available"

            # Keywords and Document Type - may not be available on Google Scholar
            publication_details['Keywords'] = "Not available on Google Scholar"
            publication_details['Document Type'] = "Not specified on Google Scholar"

            # Add to publications list
            publications.append(publication_details)
            
            # Return to the main profile page to continue with the next publication
            driver.back()
            time.sleep(2)  # Wait for the main page to reload

        # Convert the list of publication details into a DataFrame
        publications_df = pd.DataFrame(publications)
        return publications_df

    except Exception as e:
        print(f"Error retrieving detailed publications: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of error

## SJR - Retrieve SJR Metric by Journal Name

In [14]:
def get_sjr_metrics_by_name(journal_name):
    result = {
        "Journal Name": journal_name,
        "Country": "Country not found",
        "Subject Area and Category": "Subject area not found",
        "Publisher": "Publisher not found",
        "H-Index": "H-index not found",
        "Publication Type": "Publication type not found",
        "ISSN": "ISSN not found",
        "Coverage": "Coverage not found",
        "Scope": "Scope not found",
        "SJR Score": "SJR Score not found",
        "Quartile": "Quartile not found"
    }

    try:
        # Navigate to the SJR website's search page
        sjr_url = "https://www.scimagojr.com/journalsearch.php"
        driver.get(sjr_url)

        # Enter journal name in search box
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "searchinput"))
        )
        search_box.clear()
        search_box.send_keys(journal_name)
        search_box.send_keys(Keys.RETURN)

        # Wait for results and click the first result if it exists
        time.sleep(3)
        try:
            journal_link = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "span.jrnlname"))
            )
            result["Journal Name"] = journal_link.text
            journal_link.click()
        except:
            # If no journal link is found, return default 'Not found' values
            return result

        # Wait for the journal's page to load and parse it with BeautifulSoup
        time.sleep(3)
        body_html = driver.find_element(By.TAG_NAME, "body").get_attribute("outerHTML")
        soup = BeautifulSoup(body_html, 'html.parser')

        # Extract Country
        try:
            result["Country"] = soup.find("h2", string="Country").find_next("p").get_text(strip=True)
        except:
            result["Country"] = "Country not found"

        # Extract Subject Area and Category
        try:
            result["Subject Area and Category"] = soup.find("h2", string="Subject Area and Category").find_next("ul").get_text(separator=", ", strip=True)
        except:
            result["Subject Area and Category"] = "Subject area not found"

        # Extract Publisher
        try:
            result["Publisher"] = soup.find("h2", string="Publisher").find_next("p").get_text(strip=True)
        except:
            result["Publisher"] = "Publisher not found"

        # Extract H-Index
        try:
            result["H-Index"] = soup.find("h2", string="H-Index").find_next("p", class_="hindexnumber").get_text(strip=True)
        except:
            result["H-Index"] = "H-index not found"

        # Extract Publication Type
        try:
            result["Publication Type"] = soup.find("h2", string="Publication type").find_next("p").get_text(strip=True)
        except:
            result["Publication Type"] = "Publication type not found"

        # Extract ISSN
        try:
            result["ISSN"] = soup.find("h2", string="ISSN").find_next("p").get_text(strip=True)
        except:
            result["ISSN"] = "ISSN not found"

        # Extract Coverage years
        try:
            result["Coverage"] = soup.find("h2", string="Coverage").find_next("p").get_text(strip=True)
        except:
            result["Coverage"] = "Coverage not found"

        # Extract Scope
        try:
            result["Scope"] = soup.find("h2", string="Scope").find_next("div").get_text(strip=True)
        except:
            result["Scope"] = "Scope not found"

        # Extract SJR Score if available in tables
        try:
            possible_tables = soup.find_all("table")
            sjr_data = {}
            for table in possible_tables:
                if "SJR" in table.get_text() and "Year" in table.get_text():
                    rows = table.find_all("tr")[1:]  # Skip the header row
                    for row in rows:
                        columns = row.find_all("td")
                        if len(columns) >= 2:
                            year = columns[0].get_text(strip=True)
                            sjr_score = columns[1].get_text(strip=True)
                            sjr_data[year] = sjr_score
                    break

            if sjr_data:
                latest_year = max(sjr_data.keys())
                result["SJR Score"] = f"{latest_year}: {sjr_data[latest_year]}"
            else:
                result["SJR Score"] = "SJR Score not found"

        except Exception as e:
            result["SJR Score"] = "SJR Score extraction failed"

        # Extract Quartile information
        try:
            quartile_table = soup.find("table")
            for row in quartile_table.find_all("tr"):
                cells = row.find_all("td")
                if len(cells) >= 3 and cells[2].text.startswith("Q"):
                    result["Quartile"] = cells[2].text.strip()  # E.g., Q4
                    break
        except:
            result["Quartile"] = "Quartile not found"

        return result

    except Exception as e:
        print(f"An error occurred with journal '{journal_name}': {e}")
        return result  


In [15]:
def get_sjr_metrics_from_publications(publications_df):
    # Step 1: Extract unique journal names from the 'Source Title' column
    unique_journals = publications_df['Source Title'].dropna().unique()
    
    # Step 2: Initialize an empty list to hold SJR metrics data for each journal
    sjr_metrics = []

    # Step 3: Retrieve metrics for each unique journal name
    for journal in unique_journals:
        journal_metrics = get_sjr_metrics_by_name(journal)
        sjr_metrics.append(journal_metrics)

    # Step 4: Convert the list of metrics into a DataFrame
    sjr_metrics_df = pd.DataFrame(sjr_metrics)

    # Step 5: Merge publications_df with sjr_metrics_df on the journal name
    merged_df = publications_df.merge(sjr_metrics_df, left_on='Source Title', right_on='Journal Name', how='left')

    return merged_df

## Example usage

In [16]:
author_names = ["imad hafidi", "ELHADFI YOUSSEF", "ENNAJI Fatimazohra"]
for author in author_names:
    author_df, citation_df, co_authors_df = get_author_profile(author)
    publications_df = get_detailed_author_publications()  # Retrieve publication data
    merged_df = get_sjr_metrics_from_publications(publications_df)  # Merge with SJR metrics

AttributeError: 'list' object has no attribute 'replace'

In [ ]:
author_df

,Name,Affiliation,Research Interests
0,Celso Grebogi,Sixth Century Chair at University of Aberdeen,"Chaos, Fractals, Dynamical Systems, Complex Ne..."


In [ ]:
citation_df

,Metric,All Time,Since 2019
0,Citations,47737,9459
1,indice h,97,46
2,indice i10,369,189


In [ ]:
co_authors_df

,Name,Affiliation


In [ ]:
publications_df

,Title,Year,Citation Count,DOI,Authors,Source Title,Volume,Issue,Pages,Publisher,Summary,Keywords,Document Type
0,Controlling Chaos,1990,9425,10.1007/978-1-84882-523-9,"Edward Ott, Celso Grebogi, James A. Yorke",Physical Review Letters,64,Issue not found,1196-1199,American Physical Society,It is shown that one can convert a chaotic att...,Not available on Google Scholar,Not specified on Google Scholar
1,"Crises, sudden changes in chaotic attractors, ...",1983,1938,10.1016/0167-2789(83)90126-4,"Celso Grebogi, Edward Ott, James A Yorke",Physica D: Nonlinear Phenomena,7,1-3,181-200,North-Holland,The occurrence of sudden qualitative changes o...,Not available on Google Scholar,Not specified on Google Scholar
2,Using small perturbations to control chaos,1993,1242,10.1038/363411a0,"Troy Shinbrot, Celso Grebogi, James A Yorke, E...",Source title not found,363,6428,411-417,Nature Publishing Group UK,The extreme sensitivity of chaotic systems to ...,Not available on Google Scholar,Not specified on Google Scholar
3,The control of chaos: theory and applications,2000,1230,10.1016/s0370-1573(99)00096-4,"Stefanos Boccaletti, Celso Grebogi, Y-C Lai, H...",Source title not found,329,3,103-197,North-Holland,Control of chaos refers to a process wherein a...,Not available on Google Scholar,Not specified on Google Scholar
4,Chaotic attractors in crisis,1982,1044,10.1103/physrevlett.48.1507,"Celso Grebogi, Edward Ott, James A Yorke",Physical Review Letters,48,22,1507,American Physical Society,The occurrence of sudden qualitative changes o...,Not available on Google Scholar,Not specified on Google Scholar
5,Strange attractors that are not chaotic,1984,948,10.1093/oso/9780198567905.003.00016,"Celso Grebogi, Edward Ott, Steven Pelikan, Jam...",Physica D: Nonlinear Phenomena,13,1-2,261-268,North-Holland,It is shown that in certain types of dynamical...,Not available on Google Scholar,Not specified on Google Scholar
6,Fractal basin boundaries,1985,827,10.1142/9789814360258_0009,"Steven W McDonald, Celso Grebogi, Edward Ott, ...",Physica D: Nonlinear Phenomena,17,2,125-153,North-Holland,Basin boundaries for dynamical systems can be ...,Not available on Google Scholar,Not specified on Google Scholar
7,Communicating with chaos,1993,816,10.1103/physrevlett.70.3031,"Scott Hayes, Celso Grebogi, Edward Ott",Physical review letters,70,20,3031,American Physical Society,The use of chaos to transmit information is de...,Not available on Google Scholar,Not specified on Google Scholar
8,Critical exponents for crisis-induced intermit...,1987,694,10.1103/physreva.36.5365,"Celso Grebogi, Edward Ott, Filipe Romeiras, Ja...",Physical Review A,36,11,5365,American Physical Society,We consider three types of changes that attrac...,Not available on Google Scholar,Not specified on Google Scholar
9,Controlling chaotic dynamical systems,1992,646,10.23919/acc.1991.4791549,"Filipe J Romeiras, Celso Grebogi, Edward Ott, ...",Physica D: Nonlinear Phenomena,58,1-4,165-192,North-Holland,We describe a method that converts the motion ...,Not available on Google Scholar,Not specified on Google Scholar


In [ ]:
merged_df

,Title,Year,Citation Count,DOI,Authors,Source Title,Volume,Issue,Pages,Publisher_x,...,Country,Subject Area and Category,Publisher_y,H-Index,Publication Type,ISSN,Coverage,Scope,SJR Score,Quartile
0,Controlling Chaos,1990,9425,10.1007/978-1-84882-523-9,"Edward Ott, Celso Grebogi, James A. Yorke",Physical Review Letters,64,Issue not found,1196-1199,American Physical Society,...,United StatesUniversities and research institu...,"Physics and Astronomy, Physics and Astronomy (...",American Physical Society,700,Journals,"00319007, 10797114",1958-2023,Quartiles1999200120032005200720092011201320152...,2023: 3.040,Q1
1,"Crises, sudden changes in chaotic attractors, ...",1983,1938,10.1016/0167-2789(83)90126-4,"Celso Grebogi, Edward Ott, James A Yorke",Physica D: Nonlinear Phenomena,7,1-3,181-200,North-Holland,...,Country not found,Subject area not found,Publisher not found,H-index not found,Publication type not found,ISSN not found,Coverage not found,Scope not found,SJR Score not found,Quartile not found
2,Using small perturbations to control chaos,1993,1242,10.1038/363411a0,"Troy Shinbrot, Celso Grebogi, James A Yorke, E...",Source title not found,363,6428,411-417,Nature Publishing Group UK,...,Country not found,Subject area not found,Publisher not found,H-index not found,Publication type not found,ISSN not found,Coverage not found,Scope not found,SJR Score not found,Quartile not found
3,The control of chaos: theory and applications,2000,1230,10.1016/s0370-1573(99)00096-4,"Stefanos Boccaletti, Celso Grebogi, Y-C Lai, H...",Source title not found,329,3,103-197,North-Holland,...,Country not found,Subject area not found,Publisher not found,H-index not found,Publication type not found,ISSN not found,Coverage not found,Scope not found,SJR Score not found,Quartile not found
4,Chaotic attractors in crisis,1982,1044,10.1103/physrevlett.48.1507,"Celso Grebogi, Edward Ott, James A Yorke",Physical Review Letters,48,22,1507,American Physical Society,...,United StatesUniversities and research institu...,"Physics and Astronomy, Physics and Astronomy (...",American Physical Society,700,Journals,"00319007, 10797114",1958-2023,Quartiles1999200120032005200720092011201320152...,2023: 3.040,Q1
5,Strange attractors that are not chaotic,1984,948,10.1093/oso/9780198567905.003.00016,"Celso Grebogi, Edward Ott, Steven Pelikan, Jam...",Physica D: Nonlinear Phenomena,13,1-2,261-268,North-Holland,...,Country not found,Subject area not found,Publisher not found,H-index not found,Publication type not found,ISSN not found,Coverage not found,Scope not found,SJR Score not found,Quartile not found
6,Fractal basin boundaries,1985,827,10.1142/9789814360258_0009,"Steven W McDonald, Celso Grebogi, Edward Ott, ...",Physica D: Nonlinear Phenomena,17,2,125-153,North-Holland,...,Country not found,Subject area not found,Publisher not found,H-index not found,Publication type not found,ISSN not found,Coverage not found,Scope not found,SJR Score not found,Quartile not found
7,Communicating with chaos,1993,816,10.1103/physrevlett.70.3031,"Scott Hayes, Celso Grebogi, Edward Ott",Physical review letters,70,20,3031,American Physical Society,...,Country not found,Subject area not found,Publisher not found,H-index not found,Publication type not found,ISSN not found,Coverage not found,Scope not found,SJR Score not found,Quartile not found
8,Critical exponents for crisis-induced intermit...,1987,694,10.1103/physreva.36.5365,"Celso Grebogi, Edward Ott, Filipe Romeiras, Ja...",Physical Review A,36,11,5365,American Physical Society,...,Country not found,Subject area not found,Publisher not found,H-index not found,Publication type not found,ISSN not found,Coverage not found,Scope not found,SJR Score not found,Quartile not found
9,Controlling chaotic dynamical systems,1992,646,10.23919/acc.1991.4791549,"Filipe J Romeiras, Celso Grebogi, Edward Ott, ...",Physica D: Nonlinear Phenomena,58,1-4,165-192,North-Holland,...,Country not found,Subject area not found,Publisher not found,H-index not foun

### Display the final DataFrame

In [ ]:

def create_final_dataframe(author_df, citation_df, co_authors_df, merged_df):
    # Step 1: Extract author information
    author_name = author_df['Name'].iloc[0] if 'Name' in author_df.columns else "Author Name"
    author_affiliation = author_df['Affiliation'].iloc[0] if 'Affiliation' in author_df.columns else "Affiliation"
    research_interests = author_df['Research Interests'].iloc[0] if 'Research Interests' in author_df.columns else "Research Interests"

    # Add basic author information to each row in merged_df
    merged_df['Author Name'] = author_name
    merged_df['Affiliation'] = author_affiliation
    merged_df['Research Interests'] = research_interests

    # Step 2: Add co-authors and their affiliations as a single string
    if not co_authors_df.empty:
        co_authors_combined = co_authors_df['Name'].str.cat(co_authors_df['Affiliation'], sep=" - ").str.cat(sep=", ")
        merged_df['Co-Authors'] = co_authors_combined
    else:
        merged_df['Co-Authors'] = ""  # Add empty column if no co-authors data is available

    # Step 3: Expand citation metrics to separate columns after "Co-Authors"
    citation_metrics = citation_df.set_index('Metric').T if not citation_df.empty else pd.DataFrame()
    for metric in citation_metrics.columns:
        merged_df[metric + ' All Time'] = citation_metrics.loc['All Time', metric]
        merged_df[metric + ' Since 2019'] = citation_metrics.loc['Since 2019', metric]

    # Step 4: Organize columns, only including those that exist in merged_df
    columns_order = [
        'Author Name', 'Affiliation', 'Research Interests', 'Co-Authors',  # Author info and co-authors
        'Citations All Time', 'Citations Since 2019', 'indice h All Time', 'indice h Since 2019', 'indice i10 All Time', 'indice i10 Since 2019',  # Citation metrics
        'Title', 'Year', 'Citation Count', 'DOI', 'Authors', 'Source Title', 
        'Volume', 'Issue', 'Pages', 'Publisher_x', 'Summary', 'Keywords', 'Document Type',
        'Journal Name', 'Country', 'Subject Area and Category', 'Publisher_y', 
        'H-Index', 'Publication Type', 'ISSN', 'Coverage', 'Scope', 'SJR Score', 'Quartile'
    ]
    
    # Filter `columns_order` to include only those actually in `merged_df`
    existing_columns_order = [col for col in columns_order if col in merged_df.columns]
    final_df = merged_df[existing_columns_order].copy()
    
    # Step 5: Drop any unintended columns that may have been appended in previous steps
    columns_to_exclude = [col for col in final_df.columns if 'Author H-Index' in col]
    final_df.drop(columns=columns_to_exclude, inplace=True, errors='ignore')
    
    return final_df

def save_to_csv(final_df, file_path="google_scholar_dataset.csv"):
    # Check if the file exists
    if os.path.isfile(file_path):
        # Load existing data
        existing_data = pd.read_csv(file_path)
        # Append new data
        combined_data = pd.concat([existing_data, final_df], ignore_index=True)
        # Drop duplicate entries based on unique identifiers (e.g., 'Title' and 'Author Name')
        combined_data.drop_duplicates(subset=['Title', 'Author Name'], keep='last', inplace=True)
    else:
        # If file doesn't exist, use final_df as the data to save
        combined_data = final_df

    # Save the combined data back to the CSV file
    combined_data.to_csv(file_path, index=False)
    print(f"Data successfully saved to {file_path}")

# Example usage
final_df = create_final_dataframe(author_df, citation_df, co_authors_df, merged_df)
save_to_csv(final_df, "google_scholar_dataset.csv")
display(final_df)  # Display the final structured DataFrame

Data successfully saved to google_scholar_dataset.csv


,Author Name,Affiliation,Research Interests,Co-Authors,Citations All Time,Citations Since 2019,indice h All Time,indice h Since 2019,indice i10 All Time,indice i10 Since 2019,...,Country,Subject Area and Category,Publisher_y,H-Index,Publication Type,ISSN,Coverage,Scope,SJR Score,Quartile
0,Celso Grebogi,Sixth Century Chair at University of Aberdeen,"Chaos, Fractals, Dynamical Systems, Complex Ne...",,47737,9459,97,46,369,189,...,United StatesUniversities and research institu...,"Physics and Astronomy, Physics and Astronomy (...",American Physical Society,700,Journals,"00319007, 10797114",1958-2023,Quartiles1999200120032005200720092011201320152...,2023: 3.040,Q1
1,Celso Grebogi,Sixth Century Chair at University of Aberdeen,"Chaos, Fractals, Dynamical Systems, Complex Ne...",,47737,9459,97,46,369,189,...,Country not found,Subject area not found,Publisher not found,H-index not found,Publication type not found,ISSN not found,Coverage not found,Scope not found,SJR Score not found,Quartile not found
2,Celso Grebogi,Sixth Century Chair at University of Aberdeen,"Chaos, Fractals, Dynamical Systems, Complex Ne...",,47737,9459,97,46,369,189,...,Country not found,Subject area not found,Publisher not found,H-index not found,Publication type not found,ISSN not found,Coverage not found,Scope not found,SJR Score not found,Quartile not found
3,Celso Grebogi,Sixth Century Chair at University of Aberdeen,"Chaos, Fractals, Dynamical Systems, Complex Ne...",,47737,9459,97,46,369,189,...,Country not found,Subject area not found,Publisher not found,H-index not found,Publication type not found,ISSN not found,Coverage not found,Scope not found,SJR Score not found,Quartile not found
4,Celso Grebogi,Sixth Century Chair at University of Aberdeen,"Chaos, Fractals, Dynamical Systems, Complex Ne...",,47737,9459,97,46,369,189,...,United StatesUniversities and research institu...,"Physics and Astronomy, Physics and Astronomy (...",American Physical Society,700,Journals,"00319007, 10797114",1958-2023,Quartiles1999200120032005200720092011201320152...,2023: 3.040,Q1
5,Celso Grebogi,Sixth Century Chair at University of Aberdeen,"Chaos, Fractals, Dynamical Systems, Complex Ne...",,47737,9459,97,46,369,189,...,Country not found,Subject area not found,Publisher not found,H-index not found,Publication type not found,ISSN not found,Coverage not found,Scope not found,SJR Score not found,Quartile not found
6,Celso Grebogi,Sixth Century Chair at University of Aberdeen,"Chaos, Fractals, Dynamical Systems, Complex Ne...",,47737,9459,97,46,369,189,...,Country not found,Subject area not found,Publisher not found,H-index not found,Publication type not found,ISSN not found,Coverage not found,Scope not found,SJR Score not found,Quartile not found
7,Celso Grebogi,Sixth Century Chair at University of Aberdeen,"Chaos, Fractals, Dynamical Systems, Complex Ne...",,47737,9459,97,46,369,189,...,Country not found,Subject area not found,Publisher not found,H-index not found,Publication type not found,ISSN not found,Coverage not found,Scope not found,SJR Score not found,Quartile not found
8,Celso Grebogi,Sixth Century Chair at University of Aberdeen,"Chaos, Fractals, Dynamical Systems, Complex Ne...",,47737,9459,97,46,369,189,...,Country not found,Subject area not found,Publisher not found,H-index not found,Publication type not found,ISSN not found,Coverage not found,Scope not found,SJR Score not found,Quartile not found
9,Celso Grebogi,Sixth Century Chair at University of Aberdeen,"Chaos, Fractals, Dynamical Systems, Complex Ne...",,47737,9459,97,46,369,189,...,Country not found,Subject area not found,Publisher not found,H-index not found,Publication type not found,ISSN not found,Coverage not found,Scope not found,SJR Score not found,Quartile not found


In [ ]:
driver.quit()